#Librerias

In [1]:
# Cálculo numérico y manipulación de datos
import numpy as np
import pandas as pd

# Aprendizaje automático y deep learning
import tensorflow as tf

# Procesamiento de imágenes
import cv2

# Manejo de archivos Excel
import openpyxl

# Sistema y gestión de archivos
import os
import sys

# Conectar con google drive

In [2]:
# conectar con Google Drive para acceder a archivos desde Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path="/content/drive/MyDrive/analitica 3/Aplicaciones_de_operaciones" ### ruta del repositorio en drive
os.chdir(path) ### volver la carpeta del repositorio directorio de trabajo
sys.path.append(path) ### agregarla al path, poder leer archivos de funciones propios como paquetes

In [4]:
import _funciones as fn #Llamar el archivo de funciones de este proyecto

#Despliegue

In [5]:
#Definición de rutas
path_imagenes="/content/drive/MyDrive/analitica 3/Aplicaciones_de_operaciones/Data/despliegue"
path_modelo="/content/drive/MyDrive/analitica 3/Aplicaciones_de_operaciones/Salidas/skin_cancer_model.h5"
path_resultados="/content/drive/MyDrive/analitica 3/Aplicaciones_de_operaciones/Salidas/Salida_despliegue"

In [6]:
## Definir umbrales personalizados para clasificación
umbral_inferior = 0.45
umbral_superior = 0.55

In [7]:
#### cargar datos ####
path = path_imagenes # ruta de imagenes a clasificar
x, _, files= fn.load_images(path_imagenes) #cargar datos de despliegue

x = np.array(x).astype('float32') / 255.0 #Escalar datos

### Eliminar extension a nombre de archivo
lista_numeros=[] #Lista de etiquetas actuales sin extension
lista_nombres=[] #Lista para contener los nuevos nombres

#Eliminar extension del nombre
for i in files:
    numero=i.split('.')
    lista_numeros.append(numero[0])

#Añadir etiquetas
for j in lista_numeros:
    if int(j) <= 199: #Agregar etiquetas segun rango de numeros
        lista_nombres.append("b_"+j) #Agregar etiquetas de benigno
    else:
        lista_nombres.append("m_"+j) #Agregar etiquetas de maligno


# Cargar modelo entrenado
modelo = tf.keras.models.load_model(path_modelo)

# Realizar predicciones (probabilidades)
probabilidades = modelo.predict(x)

# Clasificar según umbrales definidos para cáncer de piel
clasificaciones = [
    'Cáncer' if prob > umbral_superior else
    'No Cáncer' if prob < umbral_inferior else
    'Indeterminado'
    for prob in probabilidades.flatten()
]


# Organizar salida de datos en un dataframe
resultados={
    "Paciente": lista_nombres,
    "Clasificación": clasificaciones,
    "Probabilidad": probabilidades.flatten()
}
resultados=pd.DataFrame(resultados)

#Exportar resultados en formato xlsx
resultados.to_excel(path_resultados+'/resultados.xlsx', index=False)

100%|██████████| 400/400 [00:07<00:00, 54.43it/s] 


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 161ms/step
